In [ ]:
import sys
sys.path.append('../../')

import estraces
import scared
from kresca.plot import CurvePlot
from kresca.reverse import LeakageAssessment

## Import Traces

In [ ]:
ths = estraces.read_ths_from_ets_file('../../Traces/ETS/TinyAES_1k.ets')

In [ ]:
print(ths)

## Plot Traces

In [ ]:
#put inline=True if you want to display on notebook
CurvePlot(ths.samples[:5], inline=True) 

## Make Selection Funtions

In [ ]:
@scared.reverse_selection_function
def plaintext(plaintext):
    return plaintext

@scared.reverse_selection_function
def first_addroundkey(plaintext, key):
    return scared.aes.encrypt(plaintext, key, at_round=0, after_step=scared.aes.Steps.ADD_ROUND_KEY)

@scared.reverse_selection_function
def first_subbytes(plaintext, key):
    return scared.aes.encrypt(plaintext, key, at_round=1, after_step=scared.aes.Steps.SUB_BYTES)

@scared.reverse_selection_function
def last_addroundkey(ciphertext, key):
    return scared.aes.decrypt(ciphertext, key, at_round=0, after_step=scared.aes.Steps.ADD_ROUND_KEY)

@scared.reverse_selection_function
def last_subbytes(ciphertext, key):
    return scared.aes.decrypt(ciphertext, key, at_round=1, after_step=scared.aes.Steps.SUB_BYTES)

@scared.reverse_selection_function
def ciphertext(ciphertext):
    return ciphertext

In [ ]:
AES_selection_functions = {
    'AES Plaintext': plaintext,
    'AES First Add Round Key': first_addroundkey,
    'AES First Sub Bytes': first_subbytes,
    'AES Last Add Round Key': last_addroundkey,
    'AES Last Sub Bytes': last_subbytes,
    'AES Ciphertext': ciphertext,
}

## Make Leakage Assessment Object

In [ ]:
reverse = LeakageAssessment(
    ths=ths,
    selection_functions=AES_selection_functions,
#     distinguisher=scared.CPAReverse,
#     model=scared.HammingWeight(),
#     frame=range(1750, 2750),
#     preprocesses=[]
)

In [ ]:
reverse.run()

In [ ]:
reverse.show_result()